# Generating embeddings
**Achtung!** Whole genome sequences per sample must be generated before this notebook can be used.

In [ ]:
"""
Mount Drive onto this notebook. Note that there is a specific
file structure. In particular, we have
-- MyDrive
-- -- EVO
-- -- -- CRyPTIC_reuse_table_20231208.csv
-- -- -- h37rv_genebank_flatfile.gbff

See comments to obtain any missing files.
"""

import pickle
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

evo_general_dir = '/content/drive/MyDrive/EVO/'

samples_dir = 'vcfs/'

# obtain via wget: ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/195/955/GCA_000195955.2_ASM19595v2/
h37rv_genome_file = 'GCF_000195955.2_ASM19595v2_genomic.fna'

# obtain via wget: ftp.ebi.ac.uk/pub/databases/cryptic/release_june2022/reuse/
cryptic_general_file = 'CRyPTIC_reuse_table_20231208.csv'

# note: you can use Python variables in terminal, e.g. in the next section we use
# wget -P $output_dir $vcf_file
# where output_dir and vcf_file are variables

!pip install evo-model

import torch
from evo import Evo

!pip install gumpy

import gumpy as gp
from collections import Counter

class CustomEmbedding(torch.nn.Module):
  """
    Monkey patch to obtain Evo model embeddings,
    instead of logits.
  """
  def unembed(self, u):
    return u


class GeneEmbeddingFactory():
  """
    Just a wrapper around Evo. Because we call Evo often enough, this
    makes our lives marginally better.

    22.5 GB of GPU RAM is barely enough to perform inference on 500-length
    sequences. (Storing the model requires ~12, running inference requires ~10.)
  """
  def __init__(self,
               ref_genome, # Genome
               version='evo-1-8k-base',
               device='cuda:0',
               logits=False):

    self.ref_genome = ref_genome

    evo_model = Evo(version)
    self.model = evo_model.model
    self.tokenizer = evo_model.tokenizer

    self.device = device
    self.model.to(device)
    self.model.eval()

    if not logits:
      self.model.unembed = CustomEmbedding()


  def get_gene(self, vcf, gene_name):
    """
      Note: Unless you have a lot of RAM, it is not possible
      load 12,228 whole M. tb genomes. Hence, we work on a gene-by-gene
      basis.

      input:
      - vcf: gumpy VCFFile object, M. tb VCF
      - gene_name: string

      output:
      - gene: gumpy Gene object
    """
    genome = self.ref_genome + vcf

    return genome.build_gene(gene_name)

  def run(self,
          sequence,
          to_cpu=True):
    input_ids = torch.tensor(
        self.tokenizer.tokenize(sequence),
        dtype=torch.int,
    ).to(self.device).unsqueeze(0)

    embed, _ = self.model(input_ids) # (batch, length, embed dim)
    if to_cpu: embed = embed.to(dtype=torch.float64).detach().cpu().numpy()

    return embed

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 72.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
 

In [ ]:
"""
  To change which embeddings are being generated,
  change the following variables...
  - data_dir
  - med
"""

data_dir = 'rif_embeddings_v2/'
med = 'RIF'

unique_ids = np.load(evo_general_dir + data_dir + 'unique_ids.npy')
seqs = np.load(evo_general_dir + data_dir + 'seqs.npy')

assert len(unique_ids) == len(seqs), "There are more unique_ids than seqs, or vice versa!"

ref_genome = pickle.load(open(evo_general_dir + 'h37rv_genebank.pkl', 'rb'))
gef = GeneEmbeddingFactory(ref_genome)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

configuration_hyena.py:   0%|          | 0.00/3.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/togethercomputer/evo-1-131k-base:
- configuration_hyena.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_hyena.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [ ]:
general_df = pd.read_csv(evo_general_dir + cryptic_general_file)
general_df = general_df[['UNIQUEID', 'VCF',
                       med + '_BINARY_PHENOTYPE',
                       med + '_PHENOTYPE_QUALITY',
                       med + '_MIC']]

general_df = general_df[['UNIQUEID', med + '_BINARY_PHENOTYPE']].dropna()
seq_df = pd.DataFrame({'UNIQUEID': unique_ids, 'SEQ': seqs})
df = pd.merge(general_df, seq_df, on='UNIQUEID')

print(len(df))

12072


In [ ]:
embedding_dir = evo_general_dir + data_dir
out_sub_dir = 'embeds_2.0_singles_regenerated/'

def get_site(unique_id):
  return unique_id.split('.')[1]

for i, id in tqdm(enumerate(df['UNIQUEID'])):

  out_dir = embedding_dir + out_sub_dir + 'site_' + get_site(id) + '/'
  if not os.path.exists(out_dir):
    os.makedirs(out_dir)

  if os.path.exists(out_dir + id + '.npy'): continue

  embed = gef.run(seqs[i][200:400])

  np.save(out_dir + id + '.npy', embed[0])

12072it [25:49,  7.79it/s]


In [ ]:
# import tarfile
# import os.path

# sites = [
#     '02',
#     '03',
#     '04',
#     '05',
#     '06',
#     '08',
#     '10',
#     '11',
#     '14',
#     '17',
#     '20',
# ]

# basepath = evo_general_dir + data_dir + out_sub_dir

# def make_tarfile(output_filename, source_dir):
#     with tarfile.open(output_filename, "w:gz") as tar:
#         tar.add(source_dir, arcname=os.path.basename(source_dir))

# for s in sites:
#   make_tarfile(basepath + 'site_' + s + '.tar.gz', basepath + 'site_' + s)

In [ ]:
# to_save = df[['UNIQUEID', med + '_BINARY_PHENOTYPE']]
# to_save.to_csv(evo_general_dir + data_dir + med + '_labels.csv')